In [ ]:
import pandas as pd
import numpy as np
import os
from glob import glob
from typing import List, Dict, Optional, Tuple
from datetime import datetime, timedelta

# ---------------------- 工具函数：统一日期处理 ----------------------
def _str_to_date(date_str: str) -> datetime.date:
    return datetime.strptime(date_str, "%Y-%m-%d").date()

# ---------------------- 模块1：DataLoader（保留缓存） ----------------------
class DataLoader:
    def __init__(self, 
                 daily_data_path: str = r"D:\workspace\xiaoyao\data\widetable.parquet",
                 minutely_data_root: str = r"D:\workspace\xiaoyao\data\stock_minutely_price"):
        self.daily_data_path = daily_data_path
        self.minutely_data_root = minutely_data_root
        self.daily_df = None
        self.minutely_cache = {}
        self.all_stock_codes = [path.split("=")[-1] for path in glob(f"{minutely_data_root}/stock_code=*")]
        print(f"[DataLoader] 初始化完成：宽表路径={daily_data_path}，检测到{len(self.all_stock_codes)}只股票")

    def load_daily_data(self, start_date: Optional[str] = None, end_date: Optional[str] = None) -> pd.DataFrame:
        if self.daily_df is None:
            self.daily_df = pd.read_parquet(self.daily_data_path)
            self.daily_df['date'] = self.daily_df['date'].apply(_str_to_date)
            print(f"[DataLoader] 宽表加载完成：原始数据{len(self.daily_df)}行")
        
        filtered_df = self.daily_df.copy()
        if start_date:
            filtered_df = filtered_df[filtered_df['date'] >= _str_to_date(start_date)]
        if end_date:
            filtered_df = filtered_df[filtered_df['date'] <= _str_to_date(end_date)]
        
        print(f"[DataLoader] 筛选后数据：{start_date or '开始'}至{end_date or '结束'}，共{len(filtered_df)}行")
        return filtered_df

    def load_minutely_data(self, stock_codes: List[str], date: Optional[str] = None) -> Dict[str, pd.DataFrame]:
        result = {}
        target_date = _str_to_date(date) if date else None
        
        for code in stock_codes:
            if code in self.minutely_cache:
                df = self.minutely_cache[code]
            else:
                code_path = os.path.join(self.minutely_data_root, f"stock_code={code}", "data.parquet")
                if not os.path.exists(code_path):
                    print(f"[DataLoader] 警告：{code}的分钟K不存在（路径：{code_path}）")
                    continue
                df = pd.read_parquet(code_path)
                df['date'] = df['date'].apply(_str_to_date)
                df['time'] = pd.to_datetime(df['time']).dt.time
                self.minutely_cache[code] = df
                print(f"[DataLoader] 加载分钟K：{code}，原始数据{len(df)}行")
            
            if target_date:
                df_filtered = df[df['date'] == target_date].copy()
                print(f"[DataLoader] {code}在{date}的分钟K数据：{len(df_filtered)}行")
            else:
                df_filtered = df.copy()
            
            result[code] = df_filtered
        return result

    def clear_cache(self):
        self.minutely_cache = {}
        print(f"[DataLoader] 分钟K缓存已清空")

# ---------------------- 模块2：HotspotAnalyzer（适配numpy.ndarray） ----------------------
class HotspotAnalyzer:
    def __init__(self, daily_df: pd.DataFrame):
        self.daily_df = daily_df.copy()
        print("\n[HotspotAnalyzer] 开始解析概念数据...")
        self.daily_df['concepts'] = self.daily_df['concept_name_list'].apply(self._parse_concepts)
        
        valid_count = self.daily_df['concepts'].apply(lambda x: len(x) > 0).sum()
        print(f"[HotspotAnalyzer] 概念解析结果：有效占比{valid_count/len(self.daily_df):.2%}（{valid_count}/{len(self.daily_df)}行）")
        
        if 'pre_close' not in self.daily_df.columns:
            self.daily_df = self.daily_df.sort_values(['stock_code', 'date'])
            self.daily_df['pre_close'] = self.daily_df.groupby('stock_code')['close'].shift(1)
            print(f"[HotspotAnalyzer] 提示：无pre_close字段，已用前一日close填充")
        
        self.daily_df['pct_change'] = (self.daily_df['close'] / self.daily_df['pre_close']) - 1
        print(f"[HotspotAnalyzer] 初始化完成，数据日期范围：{self.daily_df['date'].min()}至{self.daily_df['date'].max()}")

    def _parse_concepts(self, concept_data) -> List[str]:
        if isinstance(concept_data, np.ndarray):
            return [str(c).strip() for c in concept_data if str(c).strip()]
        elif isinstance(concept_data, list):
            return [str(c).strip() for c in concept_data if str(c).strip()]
        else:
            return []

    def get_hot_industries(self, date: str, top_n: int = 5) -> List[str]:
        date_obj = _str_to_date(date)
        df_day = self.daily_df[self.daily_df['date'] == date_obj]
        if df_day.empty:
            print(f"[HotspotAnalyzer] 警告：{date}无行业数据")
            return []
        
        industry_metrics = df_day.groupby('zjw_industry_name').agg(
            avg_pct_change=('pct_change', 'mean'),
            total_volume=('volume', 'sum'),
            stock_count=('stock_code', 'nunique')
        ).reset_index()
        industry_metrics = industry_metrics[industry_metrics['stock_count'] >= 3]
        if industry_metrics.empty:
            return []
        
        industry_metrics['return_rank'] = industry_metrics['avg_pct_change'].rank(ascending=False)
        industry_metrics['volume_rank'] = industry_metrics['total_volume'].rank(ascending=False)
        industry_metrics['hot_score'] = industry_metrics['return_rank'] + industry_metrics['volume_rank']
        return industry_metrics.sort_values('hot_score').head(top_n)['zjw_industry_name'].tolist()

    def get_hot_concepts(self, date: str, top_n: int = 5) -> List[str]:
        date_obj = _str_to_date(date)
        df_day = self.daily_df[self.daily_df['date'] == date_obj]
        if df_day.empty:
            print(f"[HotspotAnalyzer] 警告：{date}无数据")
            return []
        
        concept_list = []
        for _, row in df_day.iterrows():
            if row.get('paused', 0) == 1 or len(row['concepts']) == 0:
                continue
            for concept in row['concepts']:
                concept_list.append({
                    'concept': concept,
                    'stock_code': row['stock_code'],
                    'pct_change': row['pct_change']
                })
        
        if not concept_list:
            print(f"[HotspotAnalyzer] 警告：{date}无有效概念数据")
            return []
        
        concept_df = pd.DataFrame(concept_list)
        concept_stats = concept_df.groupby('concept').agg(
            stock_count=('stock_code', 'nunique'),
            avg_pct_change=('pct_change', 'mean')
        ).reset_index()
        concept_stats = concept_stats[(concept_stats['stock_count'] >= 3) & (concept_stats['avg_pct_change'] > 0)]
        if concept_stats.empty:
            print(f"[HotspotAnalyzer] 警告：{date}无符合条件的热点概念")
            return []
        
        concept_stats['count_rank'] = concept_stats['stock_count'].rank(ascending=False)
        concept_stats['return_rank'] = concept_stats['avg_pct_change'].rank(ascending=False)
        concept_stats['hot_score'] = concept_stats['count_rank'] + concept_stats['return_rank']
        
        print(f"\n[HotspotAnalyzer] {date}候选概念Top10：")
        for _, row in concept_stats.sort_values('hot_score').head(10).iterrows():
            print(f"  {row['concept']}：{row['stock_count']}只股票，平均涨幅{row['avg_pct_change']:.2%}")
        
        return concept_stats.sort_values('hot_score').head(top_n)['concept'].tolist()

    def get_daily_hotspots(self, date: str, top_n: int = 5) -> Dict[str, List[str]]:
        return {
            'hot_industries': self.get_hot_industries(date, top_n),
            'hot_concepts': self.get_hot_concepts(date, top_n)
        }

    def get_processed_daily_df(self) -> pd.DataFrame:
        return self.daily_df.copy()

# ---------------------- 模块3：AuctionSelector（竞价筛选） ----------------------
class AuctionSelector:
    def __init__(self, data_loader: DataLoader, hotspot_analyzer: HotspotAnalyzer):
        self.data_loader = data_loader
        self.hotspot_analyzer = hotspot_analyzer
        self.processed_daily_df = hotspot_analyzer.get_processed_daily_df()
        required_auc_fields = ['auc_volume', 'auc_money']
        missing_fields = [f for f in required_auc_fields if f not in self.processed_daily_df.columns]
        if missing_fields:
            raise ValueError(f"宽表缺少必要竞价字段：{missing_fields}")
        print(f"\n[AuctionSelector] 初始化完成，已确认宽表包含竞价字段：{required_auc_fields}")

    def _get_t_plus_1_date(self, t_date: str) -> str:
        return (datetime.strptime(t_date, "%Y-%m-%d") + timedelta(days=1)).strftime("%Y-%m-%d")

    def _get_hot_candidate_stocks(self, t_date: str) -> List[str]:
        hotspots = self.hotspot_analyzer.get_daily_hotspots(t_date)
        t_date_obj = _str_to_date(t_date)
        df_t = self.processed_daily_df[self.processed_daily_df['date'] == t_date_obj]
        if df_t.empty:
            print(f"[AuctionSelector] 警告：{t_date}无日度数据，候选池为空")
            return []
        
        industry_mask = df_t['zjw_industry_name'].isin(hotspots['hot_industries'])
        concept_mask = df_t['concepts'].apply(lambda x: len(set(x) & set(hotspots['hot_concepts'])) > 0)
        candidate_stocks = df_t[industry_mask | concept_mask]['stock_code'].unique().tolist()
        print(f"[AuctionSelector] T日({t_date})热点候选股票：{len(candidate_stocks)}只")
        return candidate_stocks

    def _calc_auc_indicators(self, stock_code: str, t_date: str, t_plus_1_date: str) -> Dict:
        t_date_obj = _str_to_date(t_date)
        t_plus_1_date_obj = _str_to_date(t_plus_1_date)
        
        # 获取T日数据
        df_t = self.processed_daily_df[
            (self.processed_daily_df['stock_code'] == stock_code) &
            (self.processed_daily_df['date'] == t_date_obj)
        ]
        if df_t.empty:
            print(f"[AuctionSelector] 警告：{stock_code}在{t_date}无数据")
            return {}
        t_close = df_t['close'].iloc[0]
        t_auc_volume = df_t['auc_volume'].iloc[0]
        if t_auc_volume == 0:
            print(f"[AuctionSelector] 警告：{stock_code}在{t_date}竞价成交量为0")
            return {}
        
        # 获取T+1日数据
        df_t1 = self.processed_daily_df[
            (self.processed_daily_df['stock_code'] == stock_code) &
            (self.processed_daily_df['date'] == t_plus_1_date_obj)
        ]
        if df_t1.empty:
            print(f"[AuctionSelector] 警告：{stock_code}在{t_plus_1_date}无竞价数据")
            return {}
        
        return {
            'auction_pct': (df_t1['open'].iloc[0] / t_close) - 1,
            'volume_multiple': df_t1['auc_volume'].iloc[0] / t_auc_volume,
            't1_auc_volume': df_t1['auc_volume'].iloc[0]
        }

    def select_qualified_stocks(self, t_date: str, top_n: int = 5) -> List[Tuple[str, float, float]]:
        t_plus_1_date = self._get_t_plus_1_date(t_date)
        print(f"\n[AuctionSelector] 筛选T+1日({t_plus_1_date})竞价股票")
        
        candidate_stocks = self._get_hot_candidate_stocks(t_date)
        if not candidate_stocks:
            return []
        
        qualified_stocks = []
        max_process = min(200, len(candidate_stocks))
        for i, stock_code in enumerate(candidate_stocks[:max_process]):
            if i % 50 == 0:
                print(f"[AuctionSelector] 处理第{i+1}/{max_process}只股票：{stock_code}")
            
            indicators = self._calc_auc_indicators(stock_code, t_date, t_plus_1_date)
            if not indicators:
                continue
            
            if 0.01 <= indicators['auction_pct'] <= 0.05 and indicators['volume_multiple'] >= 2 and indicators['t1_auc_volume'] > 0:
                qualified_stocks.append((
                    stock_code,
                    round(indicators['auction_pct'], 4),
                    round(indicators['volume_multiple'], 2)
                ))
                print(f"[AuctionSelector] 筛选通过：{stock_code}，涨幅{indicators['auction_pct']:.2%}，量能{indicators['volume_multiple']:.1f}倍")
        
        qualified_stocks.sort(key=lambda x: x[1], reverse=True)
        final_stocks = qualified_stocks[:top_n]
        print(f"\n[AuctionSelector] T+1日({t_plus_1_date})筛选结果：{len(final_stocks)}只")
        return final_stocks

# ---------------------- 模块4：MinuteTracker（分钟K跟踪） ----------------------
class MinuteTracker:
    def __init__(self, data_loader: DataLoader, qualified_stocks: List[Tuple[str, float, float]]):
        self.data_loader = data_loader
        self.qualified_stocks = qualified_stocks
        self.track_results = {}
        print(f"\n[MinuteTracker] 初始化完成，待跟踪股票数量：{len(qualified_stocks)}只")

    def _calc_minute_indicators(self, stock_code: str, date: str) -> Dict:
        # 加载当日分钟K
        minutely_data = self.data_loader.load_minutely_data([stock_code], date)
        if stock_code not in minutely_data or minutely_data[stock_code].empty:
            print(f"[MinuteTracker] 警告：{stock_code}在{date}无分钟K数据")
            return {}
        df_minute = minutely_data[stock_code].copy()
        
        # 筛选开盘后30分钟（9:30-10:00）
        df_minute['datetime'] = pd.to_datetime(df_minute['date'].astype(str) + ' ' + df_minute['time'].astype(str))
        track_mask = (df_minute['datetime'].dt.hour == 9) & (df_minute['datetime'].dt.minute.between(30, 59)) | \
                     (df_minute['datetime'].dt.hour == 10) & (df_minute['datetime'].dt.minute == 0)
        df_track = df_minute[track_mask].sort_values('datetime')
        if len(df_track) < 5:
            print(f"[MinuteTracker] 警告：{stock_code}开盘后数据不足（{len(df_track)}条）")
            return {}
        
        # 计算量比（前5日平均量能）
        all_dates = sorted(self.data_loader.daily_df[self.data_loader.daily_df['stock_code'] == stock_code]['date'].unique())
        target_date_idx = all_dates.index(_str_to_date(date))
        if target_date_idx < 5:
            print(f"[MinuteTracker] 警告：{stock_code}历史数据不足5天")
            return {}
        
        prev_5_dates = [d.strftime("%Y-%m-%d") for d in all_dates[target_date_idx-5:target_date_idx]]
        prev_5_volume = []
        for d in prev_5_dates:
            prev_data = self.data_loader.load_minutely_data([stock_code], d)
            if stock_code in prev_data and not prev_data[stock_code].empty:
                prev_5_volume.append(prev_data[stock_code]['volume'].mean())
        if len(prev_5_volume) < 3:
            print(f"[MinuteTracker] 警告：{stock_code}历史量能数据不足")
            return {}
        
        # 计算核心指标
        track_avg_volume = df_track['volume'].sum() / len(df_track)
        return {
            'volume_ratio': track_avg_volume / np.mean(prev_5_volume),
            'price_strength': df_track.iloc[-1]['close'] >= df_track.iloc[0]['open'] * 1.01,
            'track_close': df_track.iloc[-1]['close'],
            'auction_close': df_track.iloc[0]['open']
        }

    def generate_buy_signals(self, t_plus_1_date: str) -> Dict[str, Dict]:
        print(f"\n[MinuteTracker] 跟踪T+1日({t_plus_1_date})开盘后量能...")
        
        for stock_code, auc_pct, vol_mult in self.qualified_stocks:
            print(f"\n[MinuteTracker] 跟踪股票：{stock_code}（竞价涨幅{auc_pct:.2%}）")
            indicators = self._calc_minute_indicators(stock_code, t_plus_1_date)
            if not indicators:
                continue
            
            # 验证买入条件
            meet_volume_ratio = indicators['volume_ratio'] >= 2.0
            meet_price_strength = indicators['price_strength']
            df_minute = self.data_loader.load_minutely_data([stock_code], t_plus_1_date)[stock_code]
            track_high = df_minute[df_minute['time'].astype(str).str.contains('09:30|10:00')]['high'].max()
            no_drop_risk = track_high <= indicators['track_close'] * 1.05
            
            if meet_volume_ratio and meet_price_strength and no_drop_risk:
                self.track_results[stock_code] = {
                    '竞价涨幅': f"{auc_pct:.2%}",
                    '开盘后量比': f"{indicators['volume_ratio']:.2f}",
                    '价格强度': f"{(indicators['track_close']/indicators['auction_close']-1):.2%}",
                    '买入信号': True
                }
                print(f"[MinuteTracker] 买入信号触发：{stock_code}，量比{indicators['volume_ratio']:.2f}")
            else:
                self.track_results[stock_code] = {
                    '买入信号': False,
                    '原因': f"量比不达标({indicators['volume_ratio']:.2f})" if not meet_volume_ratio else
                           f"价格强度不足" if not meet_price_strength else
                           f"冲高回落风险"
                }
        
        buy_stocks = {k: v for k, v in self.track_results.items() if v['买入信号']}
        print(f"\n[MinuteTracker] T+1日买入信号：{len(buy_stocks)}只股票")
        return buy_stocks

# ---------------------- 模块5：ProfitCalculator（收益计算，关键补充） ----------------------
class ProfitCalculator:
    def __init__(self, data_loader: DataLoader, buy_signals: Dict[str, Dict]):
        self.data_loader = data_loader
        self.buy_signals = buy_signals
        print(f"\n[ProfitCalculator] 初始化完成，待计算收益股票数量：{len(buy_signals)}只")

    def calculate_profit(self, date: str) -> List[Dict]:
        profit_results = []
        date_obj = _str_to_date(date)
        
        for stock_code in self.buy_signals.keys():
            # 获取买入价（9:32收盘价）
            minutely_data = self.data_loader.load_minutely_data([stock_code], date)
            if stock_code not in minutely_data or minutely_data[stock_code].empty:
                print(f"[ProfitCalculator] 警告：{stock_code}在{date}无分钟K数据")
                continue
            
            df_minute = minutely_data[stock_code].copy()
            df_minute['datetime'] = pd.to_datetime(df_minute['date'].astype(str) + ' ' + df_minute['time'].astype(str))
            buy_mask = (df_minute['datetime'].dt.hour == 9) & (df_minute['datetime'].dt.minute == 32)
            if not df_minute[buy_mask].empty:
                buy_price = df_minute[buy_mask].iloc[0]['close']
            else:
                # 降级为9:31收盘价
                buy_price = df_minute[df_minute['datetime'].dt.minute == 31].iloc[0]['close']
            
            # 获取当日收盘价
            df_daily = self.data_loader.daily_df[
                (self.data_loader.daily_df['stock_code'] == stock_code) &
                (self.data_loader.daily_df['date'] == date_obj)
            ]
            if df_daily.empty:
                print(f"[ProfitCalculator] 警告：{stock_code}在{date}无收盘价")
                continue
            close_price = df_daily['close'].iloc[0]
            
            # 计算收益
            relative_profit = (close_price / buy_price - 1) * 100
            profit_results.append({
                '股票代码': stock_code,
                '买入价': round(buy_price, 2),
                '收盘价': round(close_price, 2),
                '相对收益(%)': round(relative_profit, 2),
                '是否盈利': 1 if relative_profit > 0 else 0
            })
        
        return profit_results

# ---------------------- 模块6：StrategyOptimizer（策略优化） ----------------------
class StrategyOptimizer:
    def __init__(self, data_loader: DataLoader):
        self.data_loader = data_loader
        self.hotspot_analyzer = None
        self.all_backtest_results = []
        print(f"[StrategyOptimizer] 初始化完成，数据覆盖日期：{data_loader.daily_df['date'].min()}至{data_loader.daily_df['date'].max()}")

    def _init_hotspot_analyzer(self, daily_df: pd.DataFrame):
        if self.hotspot_analyzer is None:
            self.hotspot_analyzer = HotspotAnalyzer(daily_df)

    def _is_trading_day(self, date_str: str) -> bool:
        """判断是否为交易日（宽表中有数据则视为交易日）"""
        date_obj = _str_to_date(date_str)
        return not self.data_loader.daily_df[self.data_loader.daily_df['date'] == date_obj].empty

    def single_day_backtest(self, t_date: str, **kwargs) -> Dict:
        t_plus_1_date = self._get_t_plus_1_date(t_date)
        print(f"\n[StrategyOptimizer] 单日回测：T日={t_date}，T+1日={t_plus_1_date}")
        
        # 过滤非交易日
        if not self._is_trading_day(t_date) or not self._is_trading_day(t_plus_1_date):
            result = {'交易日期': t_plus_1_date, '状态': '非交易日', '备注': 'T日或T+1日无数据'}
            print(f"[StrategyOptimizer] 回测结果：{result['状态']}")
            self.all_backtest_results.append(result)
            return result

        # 初始化热点分析器
        self._init_hotspot_analyzer(self.data_loader.daily_df)

        # 竞价筛选
        auction_selector = AuctionSelector(self.data_loader, self.hotspot_analyzer)
        qualified_stocks = []
        candidate_stocks = auction_selector._get_hot_candidate_stocks(t_date)
        if not candidate_stocks:
            result = {'交易日期': t_plus_1_date, '状态': '无候选股票'}
            self.all_backtest_results.append(result)
            return result

        # 筛选合格股票
        for stock_code in candidate_stocks[:200]:
            indicators = auction_selector._calc_auc_indicators(stock_code, t_date, t_plus_1_date)
            if not indicators:
                continue
            if kwargs['auc_pct_min'] <= indicators['auction_pct'] <= kwargs['auc_pct_max'] and \
               indicators['volume_multiple'] >= kwargs['auc_vol_mult_min'] and \
               indicators['t1_auc_volume'] > 0:
                qualified_stocks.append((stock_code, indicators['auction_pct'], indicators['volume_multiple']))
        
        if not qualified_stocks:
            result = {'交易日期': t_plus_1_date, '状态': '无合格竞价股票'}
            self.all_backtest_results.append(result)
            return result

        # 分钟跟踪
        minute_tracker = MinuteTracker(self.data_loader, qualified_stocks)
        buy_signals = minute_tracker.generate_buy_signals(t_plus_1_date)
        if not buy_signals:
            result = {'交易日期': t_plus_1_date, '状态': '无买入信号'}
            self.all_backtest_results.append(result)
            return result

        # 收益计算
        profit_calc = ProfitCalculator(self.data_loader, buy_signals)
        profit_results = profit_calc.calculate_profit(t_plus_1_date)
        if not profit_results:
            result = {'交易日期': t_plus_1_date, '状态': '无收益数据'}
            self.all_backtest_results.append(result)
            return result

        # 统计单日结果
        df_profit = pd.DataFrame(profit_results)
        total_count = len(df_profit)
        win_count = df_profit['是否盈利'].sum()
        result = {
            '交易日期': t_plus_1_date,
            '状态': '完成',
            '总交易次数': total_count,
            '盈利次数': win_count,
            '胜率(%)': round((win_count / total_count) * 100, 2) if total_count > 0 else 0,
            '平均收益(%)': round(df_profit['相对收益(%)'].mean(), 2) if total_count > 0 else 0,
            '最大收益(%)': round(df_profit['相对收益(%)'].max(), 2) if total_count > 0 else 0,
            '最大亏损(%)': round(df_profit['相对收益(%)'].min(), 2) if total_count > 0 else 0
        }
        print(f"[StrategyOptimizer] 单日回测结果：胜率{result['胜率(%)']}%，平均收益{result['平均收益(%)']}%")
        self.all_backtest_results.append(result)
        return result

    def _get_t_plus_1_date(self, t_date: str) -> str:
        return (datetime.strptime(t_date, "%Y-%m-%d") + timedelta(days=1)).strftime("%Y-%m-%d")

    def multi_day_backtest(self, start_t_date: str, end_t_date: str, **kwargs) -> Tuple[List[Dict], Dict]:
        print(f"\n[StrategyOptimizer] 多日回测：T日范围={start_t_date}至{end_t_date}")
        
        # 生成过滤后的T日列表（仅保留交易日）
        all_dates = sorted(self.data_loader.daily_df['date'].unique())
        start_date_obj = _str_to_date(start_t_date)
        end_date_obj = _str_to_date(end_t_date)
        t_date_list = [d.strftime("%Y-%m-%d") for d in all_dates 
                      if start_date_obj <= d <= end_date_obj and self._is_trading_day(d.strftime("%Y-%m-%d"))]
        
        if not t_date_list:
            return [], {'提示': '无符合条件的交易日'}

        # 逐个T日回测
        for t_date in t_date_list:
            self.single_day_backtest(t_date, **kwargs)

        # 整体统计
        df_all = pd.DataFrame([r for r in self.all_backtest_results if r['状态'] == '完成'])
        if df_all.empty:
            return self.all_backtest_results, {'提示': '无完成的回测数据'}
        
        total_trade_days = len(df_all)
        total_trades = df_all['总交易次数'].sum()
        total_wins = df_all['盈利次数'].sum()
        
        overall_stats = {
            '回测T日范围': f"{start_t_date}至{end_t_date}",
            '总T日数量': len(t_date_list),
            '有效交易天数': total_trade_days,
            '总交易次数': total_trades,
            '整体胜率(%)': round((total_wins / total_trades) * 100, 2) if total_trades > 0 else 0,
            '日均收益(%)': round(df_all['平均收益(%)'].mean(), 2),
            '收益标准差(%)': round(df_all['平均收益(%)'].std(), 2)
        }

        print(f"\n[StrategyOptimizer] 多日回测整体统计：")
        for key, value in overall_stats.items():
            print(f"  {key}：{value}")

        return self.all_backtest_results, overall_stats

# ---------------------- 测试多日回测 ----------------------
if __name__ == "__main__":
    # 1. 加载数据
    data_loader = DataLoader()
    daily_df_raw = data_loader.load_daily_data(start_date="2025-01-10", end_date="2025-01-20")
    
    # 2. 多日回测
    optimizer = StrategyOptimizer(data_loader)
    backtest_results, overall_stats = optimizer.multi_day_backtest(
        start_t_date="2025-01-10",
        end_t_date="2025-01-20",
        auc_pct_min=0.01,
        auc_pct_max=0.04,
        auc_vol_mult_min=2.0,
        minute_buy_minute=32,
        minute_volume_ratio_min=2.0,
        minute_price_strength_min=0.008
    )
    
    # 3. 输出详情
    if backtest_results:
        df_backtest = pd.DataFrame(backtest_results)
        print(f"\n=== 多日回测详情 ===")
        print(df_backtest[['交易日期', '状态', '总交易次数', '胜率(%)', '平均收益(%)']])
    
    # 4. 清空缓存
    data_loader.clear_cache()